# 신경망 학습
## 데이터에서 학습한다.
신경망의 특징은 데이터를 보고 학습할 수 있다는 점이다. 데이터에서 학습한다는 것은 가중치 매개변수의 값을 데이터를 보고 자동으로 결정한다는 뜻이다.
매개변수를 수작업으로 정한다는것은 아예 불가능하다.
데이터로부터 매개변수의 값을 정하는 방법에 대해 설명하고 MNIST 데이터셋의 손글씨 숫자를 학습하는 코드를 구현해보자.


### 데이터 주도 학습
5라는 숫자를 제대로 분류하는 프로그램을 직접 고안해 설계하기란 의외로 어려운 문제임을 알 수 있다.
사람이라면 어렵지 않게 인식하지만, 그 안에 숨은 규칙성을 명확한 로직으로 풀기가 만만치 않다.
사람마다 버릇이 달라 5를 특징짓는 규칙을 찾기도 쉽지 않고 시간도 오래 걸릴 것 같다는 느낌이 든다.

이미지에서 특징을 추출하고 그 특징의 패턴을 기계학습 기술로 학습하는 방법이 있다.
여기서 말하는 특징은 입력 데이터(입력이미지)에서 본질적인 데이터(중요한 데이터)를 정확하게 추출할 수 있도록 설계된 변환기를 가리킨다.
이미지의 특징은 보통 벡터로 기술하고, 컴퓨터 비전 분야에서는 SIFT, SURF, HOG 등의 특징을 많이 사용한다.
이런 특징을 사용하여 이미지 데이터를 벡터로 변환하고, 변환된 벡터를 가지고 지도 학습 방식의 대표 분류 기법인 SVM,KNN 등으로 학습할 수 있다.

이와 같은 기계학습에서는 모아진 데이터로부터 규칙을 찾아내는 역할을 기계가 담당한다.
무로부터 알고리즘을 설계하는 것보다 효율이 높아 문제를 해결해야 하는 사람의 부담도 덜어준다.
다만, 이미지를 벡터로 변환할 때 사용하는 특징은 여전히 사람이 설계하는 것임에 주의해야 한다.
이 말은 문제에 적합한 특징을 쓰지 않으면(혹은 특징을 설계하지 않으면) 좀 처럼 좋은 결과를 얻을 수 없다는 뜻이다.
![nn](../images/chap4/image1.png)

위의 그림과 같이 신경망은 이미지를 있는 그대로 학습한다. 두 번째 접근 방식에서는 특징을 사람이 설계했지만, 신경망은 이미지에 포함된 중요한 특징까지도 기계가 스스로 학습할 것이다.

### 훈련 데이터와 시험 데이터
기계학습 문제는 데이터를 훈련데이터와 시험데이터로 나눠 학습과 실험을 수행하는 것이 일반적이다.
우선 훈련 데이터만 사용하여 학습하면서 최적의 매개변수를 찾는다.
그런 다음 시험 데이터를 사용하여 앞서 훈련한 모델의 실력을 평가하는 것이다.
훈련 데이터와 시험데이터를 나누는 이유는 우리가 원하는 것은 범용적으로 사용할수 있는 모델이기 때문이다.
이 범용 능력을 제대로 평가하기 위해 훈련 데이터와 시험 데이터를 분리하는것이다.

### 손실 함수
신경망 학습에서는 현재의 상태를 하나의 지표로 표현한다. 그리고 그 지표를 가장 좋게 만들어주는 가중치 매개변수의 값을 탐색하는 것이다.
하나의 지표를 기준으로 최적의 매개변수 값을 탐색한다. 신경망 학습에서 사용하는 지표는 손실함수라고 한다.
손실함수는 임의의 함수를 사용할 수도 있지만 일반적으로는 오차제곱합과 교차 엔트로피 오차를 사용한다.

> 손실함수는 신경망 성능의 나쁨을 나타내는 지표로, 현재의 신경망이 훈련 데이터를 얼마나 잘 처리하지 못하느냐를 나타낸다.
성능 나쁨을 지표로 한다니 무언가 부자연스럽다고 생각할지 모르지만, 손실함수에 마이너스만 곱하면 얼마나 나쁘지 않나, 즉 얼마나 좋으냐라는 지표로 변신한다.


#### 오차 제곱합
$$ E = \frac{1}{2} \sum_k(y_k-t_k)^2 $$
여기서 $y_k$ 는 신경망의 출력(신경망이 추정한 값), $t_k$ 는 정답 레이블, $k$ 는 데이터의 차원 수를 나타낸다.

In [2]:
import numpy as np
def sum_squares_error(y,t):
    return 0.5 * np.sum((y-t) ** 2)

In [4]:
t = [0,0,1,0,0,0,0,0,0,0] # 정답은 2
# 예 1 : '2' 일 확률이 가장 높다고 추정함(0.6)
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]

sum_squares_error(np.array(y),np.array(t))
# 예 2 : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
sum_squares_error(np.array(y),np.array(t))

0.5975

#### 교차 엔트로피 오차
또 다른 손실함수로써 교차 엔트로피 오차도 자주 이용한다.
$$ E = -\sum_k t_k log_e y_k $$
$y_k$ 는 신경망의 출력, $t_k$ 는 정답 레이블이다.
또 $t_k$ 는 정답에 해당하는 인덱스의 원소만 1이고 나머지는 0이다.(원-핫 인코딩)
실질적으로 정답일 때의 추정($t_k$가 1일때의 $y_k$)의 자연로그를 계산하는 식이 된다.
예를들어 정답레이블은 2가 정답이라하고 이때의 신경망 출력이 0.6 이라면 교차 엔트로피 오차는 0.51이 된다.

In [5]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [7]:
t = [0,0,1,0,0,0,0,0,0,0] # 정답은 2
# 예 1 : '2' 일 확률이 가장 높다고 추정함(0.6)
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
cross_entropy_error(np.array(y),np.array(t))

# 예 2 : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
cross_entropy_error(np.array(y),np.array(t))

2.302584092994546